In [1]:
from langchain_groq import ChatGroq


In [3]:
llm = ChatGroq(
    temperature = 0,
    groq_api_key='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',
    model_name = 'llama-3.3-70b-versatile'
)
response = llm.invoke("the First person to land on moon was..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [14]:
#webscraping(webloader)
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.ml-jobs.ai/mozilla.ai/machine-learning-engineer-in-chicago")#need-to-be-changed
page_data = loader.load().pop().page_content
print(page_data)

Machine Learning Engineer in ChicagoBlogs Pricing MVP Testimonials About News LoginLoginBlogsPricingMVPTestimonialsAboutNewsMachine Learning Engineer in ChicagoMozilla.ai Full Time  168 days ago ApplyJob DescriptionJob Title: Machine Learning Engineer (Remote)Company: Mozilla.aiJob Type: Full-TimeExperience: Strong background in machine learning and software engineeringSalary: Competitive salary, with generous performance-based bonus plansLocation: Remote (Europe, East Coast of USA, Canada)About Mozilla.ai:Mozilla.ai is pioneering the Artificial Intelligence revolution with a commitment to open-source principles. Our mission is to empower developers to build scalable and trustworthy AI solutions. Through our Lumigator model selection platform and open-source AI Hub, we drive innovation and responsible AI practices. Join us in shaping a future of AI defined by user agency, trustworthiness, and transparency.Position Overview:We are looking for a skilled Machine Learning Engineer to join 

In [22]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ###SCRAPED DATA
    {page_data}
    ### INSTRUCTION
    the scraped text is from the carrer's page of a website.
    your job is to extract the job postings and retur them in JSON format containing 
    following keys:'role','experience','skills' and 'description'.
    Only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Machine Learning Engineer",
  "experience": "Strong background in machine learning and software engineering",
  "skills": [
    "Machine Learning Expertise",
    "Open Source Experience",
    "Deep Learning Frameworks",
    "Data Processing",
    "Experiment Management",
    "Python Proficiency",
    "Cloud Experience"
  ],
  "description": "We are looking for a skilled Machine Learning Engineer to join our remote team. This role involves working on our open-source LLM evaluation platform, focusing on model validation, selection, and deployment. You will develop tools for model management, collaborate on machine learning pipelines, and engage in end-to-end product engineering."
}
```


In [23]:
type(res.content)

str

#### convert the res.content type into json/dict

In [24]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Machine Learning Engineer',
 'experience': 'Strong background in machine learning and software engineering',
 'skills': ['Machine Learning Expertise',
  'Open Source Experience',
  'Deep Learning Frameworks',
  'Data Processing',
  'Experiment Management',
  'Python Proficiency',
  'Cloud Experience'],
 'description': 'We are looking for a skilled Machine Learning Engineer to join our remote team. This role involves working on our open-source LLM evaluation platform, focusing on model validation, selection, and deployment. You will develop tools for model management, collaborate on machine learning pipelines, and engage in end-to-end product engineering.'}

In [27]:
type(json_res)

dict

In [30]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [35]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [36]:
links = collection.query(query_texts=['Experience in python','expertise in react navtive'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [40]:
job

{'role': 'Machine Learning Engineer',
 'experience': 'Strong background in machine learning and software engineering',
 'skills': ['Machine Learning Expertise',
  'Open Source Experience',
  'Deep Learning Frameworks',
  'Data Processing',
  'Experiment Management',
  'Python Proficiency',
  'Cloud Experience'],
 'description': 'We are looking for a skilled Machine Learning Engineer to join our remote team. This role involves working on our open-source LLM evaluation platform, focusing on model validation, selection, and deployment. You will develop tools for model management, collaborate on machine learning pipelines, and engage in end-to-end product engineering.'}

In [37]:
job = json_res
job['skills']

['Machine Learning Expertise',
 'Open Source Experience',
 'Deep Learning Frameworks',
 'Data Processing',
 'Experiment Management',
 'Python Proficiency',
 'Cloud Experience']

In [39]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [ ]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

In [43]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Shruti, a Business Development Executive at AtliQ. AtliQ is an AI & Software Consulting company that specializes in helping businesses integrate automated tools to optimize processes, reduce costs, and improve efficiency. Your task is to write a professional and persuasive cold email to the client regarding the job mentioned above. 

    In the email:
    1. Introduce AtliQ and its expertise in AI and software consulting.
    2. Highlight how AtliQ can address the specific needs mentioned in the job description.
    3. add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    4. End with a call-to-action, inviting the client to discuss how AtliQ can help them achieve their goals.

    Keep the tone professional, concise, and tailored to the client's needs. Do not provide a preamble.

    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Machine Learning Solutions for Your LLM Evaluation Platform

Dear Hiring Manager,

I am Shruti, a Business Development Executive at AtliQ, an AI & Software Consulting company that specializes in helping businesses integrate automated tools to optimize processes, reduce costs, and improve efficiency. Our team of experts has a strong background in machine learning and software engineering, making us an ideal fit for your Machine Learning Engineer role.

We understand that you are looking for a skilled professional to work on your open-source LLM evaluation platform, focusing on model validation, selection, and deployment. At AtliQ, we have extensive experience in developing tools for model management, collaborating on machine learning pipelines, and engaging in end-to-end product engineering. Our expertise in machine learning, deep learning frameworks, and data processing can help you achieve your goals.

Our portfolio showcases our capabilities in machine learning and Py

{
  "role": "Machine Learning Engineer (Remote)",
  "description": "Mozilla.ai is seeking a Machine Learning Engineer to join their remote team. The role involves working on an open-source LLM evaluation platform, focusing on model validation, selection, and deployment. Responsibilities include developing tools for model management, building machine learning pipelines, and contributing to end-to-end product engineering. The position offers a competitive salary, performance-based bonuses, and a range of benefits.",
  "skills": [
    "Machine learning and software engineering",
    "Open-source development",
    "Deep learning frameworks (e.g., PyTorch)",
    "Large-scale dataset processing and data augmentation",
    "Experiment management (e.g., Weights and Biases, MLFlow, KubeFlow)",
    "Python programming and relevant libraries",
    "Cloud technologies and containerization",
    "Resource management",
    "Effective communication with non-technical audiences",
    "Problem-solving and delivering scalable solutions",
    "MLOps best practices",
    "Model evaluation and performance assessment",
    "Collaboration with product management and platform engineering",
    "End-to-end product lifecycle contribution"
  ]
}